In [1]:
%pylab
%matplotlib inline
import scipy.linalg as sla

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [27]:
def rand_dist(n):
    dist = np.random.sample(n)
    return dist / sum(dist)

def rand_sto_mat(n):
    '''Generates a random column-stochastic nxn matrix'''
    return np.array([rand_dist(n) for _ in range(n)]).T

def norm(vec, p=2):
    '''Normalizes vec using the L-p norm'''
    return vec / np.linalg.norm(vec, 1)

def get_nth_eigens(m, n=0, p=1):
    '''Returns n-th largest eigenvector and eigenvalue, L-p normalized.'''
    evals, evecs = sla.eig(m, right=True)
    index, evl = sorted(enumerate(evals), key=lambda x: abs(x[1]), reverse=True)[n]
    ev = np.real(evecs[:,index])
    return evl, norm(ev / ev[0])

def power_iter(m, niters=100):
    v  = rand_dist(np.shape(m)[0])
    for _ in range(niters):
        v = norm(m.dot(v))
    return v

def nth_eigen_iter(m, n=0, niters=100):
    '''Find the n-th highest eigenvector using power iteration'''
    ev = power_iter(m)
    evl = np.mean(m.dot(ev) / ev)
    for _ in range(n):
        m = m - evl * ev.dot(ev.T)
        ev = power_iter(m)
        evl = np.mean(m.dot(ev) / ev)
    return evl, norm(ev / ev[0])

In [3]:
# Test computing first eigenvalue 
m = rand_sto_mat(500)
%time evl, ev = get_nth_eigens(m)
%time pevl, pev = nth_eigen_iter(m)
# print(evl, ev)
# print(pev)
assert np.linalg.norm(pev - ev) < 1e-15, np.linalg.norm(pev - ev)

CPU times: user 900 ms, sys: 3.33 ms, total: 903 ms
Wall time: 928 ms
CPU times: user 30 ms, sys: 0 ns, total: 30 ms
Wall time: 30.3 ms


In [7]:
# Timing tests
%time _, __ = nth_eigen_iter(rand_sto_mat(500))
%time _, __ = nth_eigen_iter(rand_sto_mat(1000))
%time _, __ = nth_eigen_iter(rand_sto_mat(1500))
%time _, __ = nth_eigen_iter(rand_sto_mat(2000))
%time _, __ = nth_eigen_iter(rand_sto_mat(2500))


Wall time: 4 s



Wall time: 1.02 s
CPU times: user 3.93 s, sys: 66.7 ms, total: 3.99 s


Wall time: 511 ms
CPU times: user 1.02 s, sys: 0 ns, total: 1.02 s


Wall time: 293 ms
CPU times: user 510 ms, sys: 0 ns, total: 510 ms

CPU times: user 40 ms, sys: 0 ns, total: 40 ms
Wall time: 37.5 ms
CPU times: user 293 ms, sys: 0 ns, total: 293 ms

In [66]:
# Test computing second eigenvalue
lst = []
for _ in range(1000):
    m = rand_sto_mat(10)
    evl, ev = get_nth_eigens(m, n=1)
    pevl, pev = nth_eigen_iter(m, n=1,niters=500)
    lst.append(np.linalg.norm(ev-pev))
plt.hist(lst)

(array([ 546.,   86.,   47.,   88.,   49.,   46.,   46.,   47.,   34.,   11.]),
 array([  1.36592263e-16,   8.95946693e-02,   1.79189339e-01,
          2.68784008e-01,   3.58378677e-01,   4.47973347e-01,
          5.37568016e-01,   6.27162685e-01,   7.16757355e-01,
          8.06352024e-01,   8.95946693e-01]),
 <a list of 10 Patch objects>)